In [1]:
#いろいろインポート
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt
import seaborn
from pandas import DataFrame
from tqdm import tqdm
from imblearn.under_sampling import RandomUnderSampler #アンダーサンプリング用
import pickle
# 機械学習用
from sklearn.cluster import KMeans #クラスタリング用
from sklearn.ensemble import RandomForestClassifier#ランダムフォレスト
from copy import deepcopy as cp
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression as LR

pd.set_option('display.width',400)#勝手に改行コードを入れられるのを防ぐ
place='omura'

In [2]:
#モデルスコアデータの読み込み
score_df = pd.read_csv('{}_model_score.csv'.format(place))
score_df=score_df.drop(["Unnamed: 0"],axis=1)#csvファイルについている名無しの列を削除
display(score_df)

,target_com,depth,target_per,総収益,投資金額,出現数,購買予測数,利益率,購買的中率,的中数
0,1.0,4.0,51.0,291960.0,314800.0,312.0,3148.0,92.744600,9.911055,312.0
1,1.0,4.0,52.0,291960.0,314800.0,312.0,3148.0,92.744600,9.911055,312.0
2,1.0,4.0,53.0,291960.0,314800.0,312.0,3148.0,92.744600,9.911055,312.0
3,1.0,4.0,54.0,291960.0,314800.0,312.0,3148.0,92.744600,9.911055,312.0
4,1.0,4.0,55.0,291960.0,314800.0,312.0,3148.0,92.744600,9.911055,312.0
...,...,...,...,...,...,...,...,...,...,...
56920,120.0,8.0,145.0,13550.0,39500.0,1.0,395.0,34.303797,0.253165,1.0
56921,120.0,8.0,146.0,13550.0,39500.0,1.0,395.0,34.303797,0.253165,1.0
56922,120.0,8.0,147.0,13550.0,39500.0,1.0,395.0,34.303797,0.253165,1.0
56923,120.0,8.0,148.0,13550.0,39500.0,1.0,395.0,34.303797,0.253165,1.0


#### モデル性能評価用のデータフレーム作成(result_comごとの閾値を持つ)


In [3]:
#学習データの読み込み
result_df = pd.read_csv('../csv/train_{}.csv'.format(place))
result_df=result_df.drop(["Unnamed: 0"],axis=1)#csvファイルについている名無しの列を削除

df=result_df
#学習、テストデータ切り分け
num_data = len(df) 
num_valid = int(num_data/6.0)
valid_df = df.loc[(num_data-num_valid):num_data, :]#なるべく最新のデータを未知データに
train_df = df.loc[0:(num_data-num_valid-1), :]
train_money=pd.Series(train_df['money'])
valid_money=pd.Series(valid_df['money'])
#出現数の分布
result_com_s=valid_df['result_com'].value_counts()
result_com_s=result_com_s.sort_index()
gain_mean=valid_df.groupby('result_com')['money'].mean()
gain_mean=gain_mean.sort_index()

gain_median=valid_df.groupby('result_com')['money'].median()
gain_median=gain_median.sort_index()
result_com_df=pd.DataFrame({'result_com':result_com_s.index,
                            'result_com_num':result_com_s.values,
                            'result_com_per':result_com_s.values/sum(result_com_s.values)*100,
                            'gain_mean':gain_mean.values,
                            'gain_median':gain_median.values,})
display(result_com_df)

,result_com,result_com_num,result_com_per,gain_mean,gain_median
0,1.0,312,9.911055,935.769231,720.0
1,2.0,190,6.035578,1100.263158,830.0
2,3.0,151,4.796696,1477.615894,1070.0
3,4.0,88,2.795426,2530.454545,1570.0
4,5.0,221,7.020330,1199.321267,1010.0
...,...,...,...,...,...
110,114.0,3,0.095299,25233.333333,18970.0
111,115.0,1,0.031766,9980.000000,9980.0
112,117.0,3,0.095299,10496.666667,4630.0
113,118.0,1,0.031766,11810.000000,11810.0


### モデル性能評価機能

In [4]:
#基準をクリアしたモデルを格納するdf
depths_arr=[4,5,6,7,8]
model_para_df=pd.DataFrame(columns=['target_com','depth','target_per','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])
for target_com in result_com_df['result_com'].values:
    target_com_df=score_df[score_df['target_com']==target_com]
    #閾値の作成===============================================================
    gain_th=110#利益率の閾値
    result_s=result_com_df[result_com_df['result_com']==target_com]
    buy_accuracy_th=result_s['result_com_per'].values[0]*1.1#買ったうちの的中率の閾値
    num_tp_th=result_s['result_com_num'].values[0]*0.2#あたった回数の閾値(出現回数の20%が的中)
    
    diff_gain_th=108#利益率の閾値
    diff_buy_accuracy_th=result_s['result_com_per'].values[0]*1.1#前後モデルの買ったうちの的中率の閾値
    diff_num_tp_th=result_s['result_com_num'].values[0]*0.13#前後モデルのあたった回数の閾値(出現回数の20%が的中)
    
    #==========================================================
    for target_depth in depths_arr:
        target_com_depth_df=target_com_df[target_com_df['depth']==target_depth]
        
        len_df=len(target_com_depth_df)
        df=target_com_depth_df.copy()
        df['number_i']=np.arange(0,len_df,1)
        
        for _, row in df.iterrows():
            if ((row['number_i']==0) or (row['number_i']==1) or (row['number_i']==len_df-2) or (row['number_i']==len_df-1)):
                pass
            else:
                #前後のモデルのスコア===================================
                diff_m2_row=df[df['number_i']==(row['number_i']-2)]
                diff_m1_row=df[df['number_i']==(row['number_i']-1)]
                diff_p1_row=df[df['number_i']==(row['number_i']+1)]
                diff_p2_row=df[df['number_i']==(row['number_i']+2)]
                diff_models=[diff_m2_row,diff_m1_row,diff_p1_row,diff_p2_row]
                #===================================================
                if ((row['利益率']>gain_th) and (row['購買的中率']>buy_accuracy_th) and (row['的中数']>=num_tp_th)):#初めに真ん中のモデルの性能評価
                    #前後モデルの評価
                    flag_arr=[0,0,0,0]
                    for index in range(len(flag_arr)):

                        diff_model=diff_models[index].iloc[0,:]
                        if ((diff_model['利益率']>diff_gain_th) and (diff_model['購買的中率']>diff_buy_accuracy_th) and (diff_model['的中数']>=diff_num_tp_th)):#前後モデルの性能評価
                            flag_arr[index]=1
                        else:
                            pass
                    if sum(flag_arr)==4:
                        model_para_df=model_para_df.append(row)
                        
                else:
                    pass
        

In [5]:
model_para_df

,target_com,depth,target_per,総収益,投資金額,出現数,購買予測数,利益率,購買的中率,的中数,number_i
3341,7.0,7.0,125.0,26170.0,20100.0,140.0,201.0,130.199005,14.427861,29.0,74.0
3441,7.0,8.0,126.0,26070.0,23100.0,140.0,231.0,112.857143,12.554113,29.0,75.0
3443,7.0,8.0,128.0,24920.0,19100.0,140.0,191.0,130.471204,14.659686,28.0,77.0
5506,12.0,4.0,112.0,38230.0,34300.0,53.0,343.0,111.457726,3.790087,13.0,61.0
7489,16.0,4.0,115.0,46460.0,38700.0,41.0,387.0,120.051680,3.100775,12.0,64.0
...,...,...,...,...,...,...,...,...,...,...,...
52761,109.0,6.0,144.0,33410.0,28000.0,5.0,280.0,119.321429,0.714286,2.0,93.0
52762,109.0,6.0,145.0,33410.0,28000.0,5.0,280.0,119.321429,0.714286,2.0,94.0
52763,109.0,6.0,146.0,35010.0,25400.0,5.0,254.0,137.834646,1.181102,3.0,95.0
52861,109.0,7.0,145.0,33410.0,29700.0,5.0,297.0,112.491582,0.673401,2.0,94.0


In [6]:
model_para_df.to_csv('../csv/{}_good_score_model.csv'.format(place), encoding='utf_8_sig')

In [33]:
diff_model

(1047, target_com        3.000000
 depth             4.000000
 target_per      108.000000
 総収益           38850.000000
 投資金額          46200.000000
 出現数             145.000000
 購買予測数           462.000000
 利益率              84.090909
 購買的中率             9.523810
 的中数              44.000000
 number_i         57.000000
 Name: 1047, dtype: float64)

In [17]:
len_df

99

In [34]:
type(diff_model)

tuple